In [4]:
import skimage
import numpy as np
from skimage import io, transform
import os
import shutil
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import tensorflow as tf
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import urllib.request
import urllib.error

%matplotlib inline

Before opening the Jupyter Notebook make sure you have cloned the `models` folder into the repository root directory and run the following from the root diretory to install the TensorFlow API

```bash
git clone https://github.com/tensorflow/models.git
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
cd ..
cd ..
```

Set Up Path Directories
--------------------------

In [3]:
root = os.getcwd()
imagePath = os.path.join(root, 'images')
labelsPath = os.path.join(root, 'labels')
linksPath = os.path.join(imagePath, 'imageLinks')
trainPath = os.path.join(imagePath, 'train')
testPath = os.path.join(imagePath, 'test')

Create Dataset
----------------

Some very large detection datasets such as [Pascal](http://www.cs.stanford.edu/~roozbeh/pascal-context/) and [COCO](http://cocodataset.org/#home) exist already but if you want to train a custom object detection class you simply need to create and label your own dataset.
For my dataset I decided to collect images of chess pieces.  I started only using white and black pawns but I’m hoping to include all chess pieces in the future.  Since I was gathering all my images from search engines I decided to make a list of links in a text file which later can be downloaded using a script and scikit image. 

The labels for the pawn detector are all together in the labels dir and will be moved accordingly with their associated images.  If you plan on creating your own dataset make your own text file of links in the images/imageLinks folder and set copyLabels to False.

# A simple link checker that will make sure link to image is valid
def check(url):
    try:
        f= urllib.request.urlopen(url)
    except urllib.error.URLError :
            return False
    return True

'''
This function will download and resize all images in the
imageLinks folder and will split into train and test folders
with their associated label.

It is your responsibility to ensure that use of copyrighted images
accessed in connection with this script complies with any license
restrictions that may apply.
'''


copyLabels = True
trainPercent = 0.85

listing = os.listdir(linksPath) 
for classes in listing:
    os.chdir(linksPath)
    text = open(classes, 'r')
    links = text.readlines()
    links = [i.strip() for i in links]
    
    cut = int(np.floor(len(links)*trainPercent))
    
    for i in range(cut):
        os.chdir(trainPath)
        if check(links[i]):
            image = skimage.io.imread(links[i])
            image = skimage.transform.resize(image, [300,300])
            skimage.io.imsave(classes[:-4]+str(i)+'.jpg', image)
            if copyLabels:
                label = classes[:-4]+str(i)+'.xml'
                shutil.copyfile(labelsPath+'/'+label,trainPath+'/'+label) 
        
    for i in range(cut,len(links)):
        os.chdir(testPath)
        if check(links[i]):
            image = skimage.io.imread(links[i])
            image = skimage.transform.resize(image, [300,300])
            skimage.io.imsave(classes[:-4]+str(i)+'.jpg', image)
            if copyLabels:
                label = classes[:-4]+str(i)+'.xml'
                shutil.copyfile(labelsPath+'/'+label,testPath+'/'+label) 
        

Convert XML Labels to CSV
-----------------------------

In [5]:
# Modified From:
# https://github.comr/datitran/raccoon_dataset/blob/master/xml_to_csv.py

os.chdir(root)

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text+".png",
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[2][0].text),
                     int(member[2][1].text),
                     int(member[2][2].text),
                     int(member[2][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    
    for i in [trainPath, testPath]:
        image_path = i
        folder = os.path.basename(os.path.normpath(i))
        xml_df = xml_to_csv(image_path)
        xml_df.to_csv('data/'+folder+'.csv', index=None)
        print('Successfully converted xml to csv.')
    
main()

Successfully converted xml to csv.
Successfully converted xml to csv.


Create TF Record
------------------------------------------------------

When training models with TensorFlow using [tfrecords](http://goo.gl/oEyYyR) files help optimize your data feed.  We can generate a tfrecord using code adapted from this [raccoon detector](https://github.com/datitran/raccoon_dataset/blob/master/generate_tfrecord.py).

In [1]:
%%bash 

python generate_tfrecord.py
mv test.record data
mv train.record data

/home/malkiebr/.bash_profile: line 1: â€œexport: command not found
Traceback (most recent call last):
  File "generate_tfrecord.py", line 10, in <module>
    import pandas as pd
  File "/usr/local/lib/python2.7/dist-packages/pandas/__init__.py", line 23, in <module>
    from pandas.compat.numpy import *
  File "/usr/local/lib/python2.7/dist-packages/pandas/compat/numpy/__init__.py", line 24, in <module>
    'this pandas version'.format(_np_version))
ImportError: this version of pandas is incompatible with numpy < 1.9.0
your numpy version is 1.8.2.
Please upgrade numpy to >= 1.9.0 to use this pandas version
mv: cannot stat â€˜test.recordâ€™: No such file or directory
mv: cannot stat â€˜train.recordâ€™: No such file or directory


Download Model
----------------

There are [models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) in the TensorFlow API that you can use depending on your needs.  If you want a high speed model that can work on detecting video feed at high fps the [single shot detection](http://www.cs.unc.edu/%7Ewliu/papers/ssd.pdf) model works best, but you gain speed at the cost of accuracy. Some object detection models detect objects by sliding different sized boxes across the image running the classifier many time on different sections of the image, this of course can be very resource consuming.  As it’s name suggests single shot detection determines all bounding box probabilities in one go, hence why it is a vastly faster model. I’ve already configured the [config](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs) file for mobilenet and included it in the GitHub repository for this post.  Depending on your computer you may have to lower the batch size in the config file if you run out of memory.



In [1]:
%%bash

#wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_11_06_2017.tar.gz
#tar xvzf ssd_mobilenet_v1_coco_11_06_2017.tar.gz

wget http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_2018_01_28.tar.gz
tar xvzf faster_rcnn_resnet101_coco_2018_01_28.tar.gz

faster_rcnn_resnet101_coco_2018_01_28/
faster_rcnn_resnet101_coco_2018_01_28/model.ckpt.index
faster_rcnn_resnet101_coco_2018_01_28/checkpoint
faster_rcnn_resnet101_coco_2018_01_28/pipeline.config
faster_rcnn_resnet101_coco_2018_01_28/model.ckpt.data-00000-of-00001
faster_rcnn_resnet101_coco_2018_01_28/model.ckpt.meta
faster_rcnn_resnet101_coco_2018_01_28/saved_model/
faster_rcnn_resnet101_coco_2018_01_28/saved_model/saved_model.pb
faster_rcnn_resnet101_coco_2018_01_28/saved_model/variables/
faster_rcnn_resnet101_coco_2018_01_28/frozen_inference_graph.pb


/home/malkiebr/.bash_profile: line 1: â€œexport: command not found
--2018-03-24 20:52:00--  http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_2018_01_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.3.176, 2607:f8b0:400a:800::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.3.176|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 593445323 (566M) [application/x-tar]
Saving to: â€˜faster_rcnn_resnet101_coco_2018_01_28.tar.gzâ€™

     0K .......... .......... .......... .......... ..........  0% 8.52M 66s
    50K .......... .......... .......... .......... ..........  0% 5.55M 84s
   100K .......... .......... .......... .......... ..........  0% 11.5M 73s
   150K .......... .......... .......... .......... ..........  0% 11.1M 67s
   200K .......... .......... .......... .......... ..........  0% 7.95M 68s
   250K .......... .......... .......... .......... ..........  0% 14

Train Model
-------------
Since we are only retraining the last layer of our mobilenet model a high end gpu is not required (but certainly can speed things up). Training time should roughly take an hour.  It will be much easier to watch the training process if you copy and paste the following code into a new terminal in the repository root directory.  Once our loss drops to a consistant level for a good while we can stop TensorFlow training by pressing ctrl+c.

To train the model copy and paste the following code into a new terminal from the repository root directory.  If using Docker create a new terminal pressing `ctrl` + `b` then `c`.

```bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
cd ..
cd ..

python3 models/research/object_detection/train.py --logtostderr --train_dir=data/ --pipeline_config_path=data/ssd_mobilenet_v1_pets.config
```

Watch Training in TensorBoard
---------------------------------

We can use TensorBoard to monitor our total loss and other variables.  From the repository root directory run this command.

```bash
tensorboard --logdir='data'
```

Copy Object Detection Utilities to Our Root Directory
-------------------------------------------------------------

We just need to move some of the utilities from the Object Detection folder into our root path.

In [ ]:
%%bash

cp -R models/research/object_detection/utils/. utils

Export Inference Graph
-------------------------

I highly recommend you expiriment with different checkpoints as your model trains.  We can get a list of all the ckpt files with the following.

In [3]:
%%bash 
cd data
ls model*.index

model.ckpt-0.index
model.ckpt-1.index


/home/malkiebr/.bash_profile: line 1: â€œexport: command not found


You can then added the cpkt number to our trained_checkpoint argument.

In [17]:
%%bash 
rm -rf object_detection_graph
python models/research/object_detection/export_inference_graph.py --input_type image_tensor --pipeline_config_path data/ssd_mobilenet_v1_pets.config --trained_checkpoint_prefix data/model.ckpt-17323 --output_directory object_detection_graph

/home/malkiebr/.bash_profile: line 1: â€œexport: command not found
Traceback (most recent call last):
  File "models/research/object_detection/export_inference_graph.py", line 94, in <module>
    import tensorflow as tf
ImportError: No module named tensorflow


Test Model
-----------

In [5]:
# Modified From API
# https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = 'object_detection_graph/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'data/label_map.pbtxt'

NUM_CLASSES = 2

PATH_TO_TEST_IMAGES_DIR = 'images/validation/'
TEST_IMAGE_PATHS = os.listdir(PATH_TO_TEST_IMAGES_DIR)

IMAGE_SIZE = (12, 12)

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(
        np.uint8)


In [ ]:
# Modified From API
# https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name(
            'detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name(
            'detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name(
            'detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        for image_path in TEST_IMAGE_PATHS:
            image = Image.open(PATH_TO_TEST_IMAGES_DIR+image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
                [
                    detection_boxes, detection_scores, detection_classes,
                    num_detections
                ],
                feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=2)
            plt.figure(figsize=IMAGE_SIZE)
            plt.imshow(image_np)
            image.close()


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
